## Variational Quantum Deflation Tutorial

#### H2 example with overlap calculated classically and with "swap test" quantum circuit 

In [1]:
import sys

sys.path.insert(0, '/Users/shijunang/qibochem/src')

In [2]:
import numpy as np

from qibo import Circuit, gates
from qibo.optimizers import optimize
from qibochem.measurement import expectation
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, Z, X, Y

In [3]:
def swap_test_circuit(mol, ansatz, excitations):

    circuit = ansatz(mol, excitations=excitations)
    circuit.nqubits = circuit.nqubits * 2 + 1
    circuit.queue.nqubits = circuit.nqubits
    
    system_nqubits = circuit.nqubits // 2
    
    for i in range(len(circuit.raw['queue'])):
        gate_attributes = circuit.raw['queue'][i]
        
        if circuit.raw['queue'][i]['_class'] == 'H':
           circuit.add(gates.H(gate_attributes['init_args'][0] + system_nqubits))
        elif circuit.raw['queue'][i]['_class'] == 'CNOT':
           circuit.add(gates.CNOT(gate_attributes['init_args'][0] + system_nqubits, 
                                  gate_attributes['init_args'][1] + system_nqubits))
        elif circuit.raw['queue'][i]['_class'] == 'SDG':
           circuit.add(gates.SDG(gate_attributes['init_args'][0] + system_nqubits))
        elif circuit.raw['queue'][i]['_class'] == 'S':
           circuit.add(gates.S(gate_attributes['init_args'][0] + system_nqubits))
        elif circuit.raw['queue'][i]['_class'] == 'RZ':
           circuit.add(gates.RZ(gate_attributes['init_args'][0] + system_nqubits, np.random.uniform(0.0, 2*np.pi)))
        elif circuit.raw['queue'][i]['_class'] == 'X':
           circuit.add(gates.X(gate_attributes['init_args'][0] + system_nqubits))
    
    givens_circuit = givens_excitation_ansatz(h2, excitations=[[mol.nelec-1, mol.nelec]], include_hf=False, use_mp2_guess=False)
    
    for i in range(len(givens_circuit.raw['queue'])):
        gate_attributes = givens_circuit.raw['queue'][i]
        
        if givens_circuit.raw['queue'][i]['_class'] == 'H':
            circuit.add(gates.H(gate_attributes['init_args'][0] + system_nqubits))
        elif givens_circuit.raw['queue'][i]['_class'] == 'CNOT':
            circuit.add(gates.CNOT(gate_attributes['init_args'][0] + system_nqubits, 
                                         gate_attributes['init_args'][1] + system_nqubits))
        elif givens_circuit.raw['queue'][i]['_class'] == 'RY':
            circuit.add(gates.RY(gate_attributes['init_args'][0] + system_nqubits, np.random.uniform(0.0, 2*np.pi)))
        elif givens_circuit.raw['queue'][i]['_class'] == 'X':
            circuit.add(gates.X(gate_attributes['init_args'][0] + system_nqubits))
    
    circuit.add(gates.H(circuit.nqubits-1))
    for i in range(0,system_nqubits):
        circuit.add(gates.SWAP(i,i+system_nqubits).controlled_by(circuit.nqubits-1))
    circuit.add(gates.H(circuit.nqubits-1))
    
    return circuit

In [4]:
# VQD Loss Function

def vqd_loss(params, circuit, hamiltonian, vqd_beta, vqe_gs):
    
    circuit.set_parameters(params)
    result = hamiltonian.backend.execute_circuit(circuit)
    final_state = result.state()
    overlap = np.inner(vqe_gs, final_state)
    print(hamiltonian.expectation(final_state), overlap)
    return hamiltonian.expectation(final_state) + vqd_beta * np.abs(overlap) ** 2

def vqd_swap_loss(params, es_circuit, swap_circuit, hamiltonian, vqd_beta):
    
    es_circuit.set_parameters(params)
    result = hamiltonian.backend.execute_circuit(es_circuit)
    final_state = result.state()
    swap_circuit.set_parameters(params)
    swap_hamiltonian = SymbolicHamiltonian(Z(hamiltonian.nqubits*2),hamiltonian.nqubits*2+1)
    swap_result = swap_hamiltonian.backend.execute_circuit(swap_circuit)
    swap_state = swap_result.state()
    print(hamiltonian.expectation(final_state), swap_hamiltonian.expectation(swap_state))
    return hamiltonian.expectation(final_state) + vqd_beta * swap_hamiltonian.expectation(swap_state)

In [5]:
import numpy as np
from qibo.models import VQE

from qibochem.driver import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit, ucc_ansatz, givens_excitation_ansatz, givens_excitation_circuit

# Define the H2 molecule and obtain its 1-/2- electron integrals with PySCF
h2 = Molecule([('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.742))])
h2.run_pyscf()
# Generate the molecular Hamiltonian
hamiltonian = h2.hamiltonian()

# Build a UCC circuit ansatz for running VQE
circuit = ucc_ansatz(h2, excitations=[[0,1,2,3]])

# Create and run the VQE, starting with random initial parameters
vqe = VQE(circuit, hamiltonian)

initial_parameters = np.random.uniform(0.0, 2*np.pi, 8)
best, vqe_params, extra = vqe.minimize(initial_parameters)

[Qibo 0.2.16|INFO|2025-09-19 13:27:56]: Using numpy backend on /CPU:0


### VQD using quantum state vectors of GS and ES (explicit inner product method)

In [6]:
results = vqe.circuit()
vqe_gs = results.state()

In [7]:
givens_circuit = givens_excitation_ansatz(h2, excitations=[[1,2]], include_hf=False, use_mp2_guess=False)

In [8]:
# Create a concatenated circuit of UCCSD and Given's

es_circuit = circuit + givens_circuit 
all_params = np.append(vqe_params, np.random.rand(len(givens_circuit.get_parameters())))
es_circuit.set_parameters(all_params)

for count in range(len(circuit.trainable_gates)):
    del es_circuit.trainable_gates[0]

# All trainable parameters (Given's trainable, UCCSD not trainable)
es_params = all_params[len(vqe_params):]

#nparams has to be manually updated
es_circuit.trainable_gates.nparams = len(es_circuit.trainable_gates)

In [9]:
best, params, extra = optimize(vqd_loss, es_params, args=(es_circuit, hamiltonian, 100, vqe_gs), method='BFGS')

-1.1218270842683997 (0.9871401362895607+0j)
-1.1218270828480073 (0.9871401350985309+0j)
-1.1218270856887926 (0.9871401374805902+0j)
-0.7815642595373837 (0.6412087330844337+0j)
-0.7815642551084475 (0.6412087273671077+0j)
-0.7815642639663201 (0.6412087388017595+0j)
-0.9503732301340447 (-0.8310282023793597+0j)
-0.9503732259733333 (-0.831028198235121+0j)
-0.9503732342947556 (-0.8310282065235982+0j)
-0.5471296684034882 (0.15181486250023113+0j)
-0.5471296670528216 (0.15181485513601053+0j)
-0.5471296697541547 (0.15181486986445158+0j)
-0.5334189822150568 (0.018709296001474564+0j)
-0.5334189820466814 (0.018709288552197983+0j)
-0.5334189823834322 (0.018709303450751037+0j)
-0.550182052742784 (-0.16763323690042378+0j)
-0.5501820542303216 (-0.16763324424557446+0j)
-0.5501820512552468 (-0.1676332295552732+0j)
-0.533207554361528 (0.00015484375908563123+0j)
-0.5332075543601343 (0.0001548363085050728+0j)
-0.533207554362922 (0.00015485120966618964+0j)
-0.5332075398783308 (-3.6384541409967044e-08+0j)
-0.

In [10]:
es_circuit.set_parameters(params)
print(f"Expectation value: {expectation(es_circuit, hamiltonian):.8f} (Exact first excited state of H2)")

Expectation value: -0.53320754 (Exact first excited state of H2)


### VQD using swap test circuit

In [11]:
swap_circuit = swap_test_circuit(h2, ucc_ansatz, [[0,1,2,3]])
all_params = np.hstack((vqe_params, vqe_params, np.random.rand(len(givens_circuit.get_parameters())))).ravel()
# all_params = np.append(vqe_params, vqe_params)
swap_circuit.set_parameters(all_params)

for count in range(len(vqe_params)*2):
    del swap_circuit.trainable_gates[0]

#nparams has to be manually updated
swap_circuit.trainable_gates.nparams = len(swap_circuit.trainable_gates)

In [12]:
# Visualizing the SWAP circuit
swap_circuit.draw()

0:     ─X───H─────X─RZ─X─────H─SDG─H─────────X─RZ─X─────H─S─SDG─H─────X─RZ─X─ ...
1:     ─X───H───X─o────o─X───H─SDG─H───────X─o────o─X───H─S─H───────X─o────o─ ...
2:     ─SDG─H─X─o────────o─X─H─S───SDG─H─X─o────────o─X─H─S─H─────X─o──────── ...
3:     ─H─────o────────────o─H─H─────────o────────────o─H─H───────o────────── ...
4:     ────────────────────────────────────────────────────────────────────── ...
5:     ────────────────────────────────────────────────────────────────────── ...
6:     ────────────────────────────────────────────────────────────────────── ...
7:     ────────────────────────────────────────────────────────────────────── ...
8:     ────────────────────────────────────────────────────────────────────── ...

0: ... ────H─S───H─────X─RZ─X─────H─SDG─H─────X─RZ─X─────H─S───H─────────X─RZ ...
1: ... X───H─SDG─H───X─o────o─X───H─S───H───X─o────o─X───H─SDG─H───────X─o─── ...
2: ... o─X─H─H─────X─o────────o─X─H─SDG─H─X─o────────o─X─H─S───SDG─H─X─o───── ...
3: ... ──o─H─H─

In [13]:
best, params, extra = optimize(vqd_swap_loss, es_params, args=(es_circuit, swap_circuit, hamiltonian, 100), method='BFGS')
print(f"Expectation value: {expectation(es_circuit, hamiltonian):.8f} (Exact first excited state of H2)")

-1.1218270842683997 0.9744456486737707
-1.1218270828480073 0.974445646322344
-1.1218270856887926 0.9744456510251966
-0.7815642595373851 0.4111486393837461
-0.7815642551084492 0.41114863205174745
-0.7815642639663218 0.41114864671574486
-0.9503732301340511 0.69060787314988
-0.95037322597334 0.6906078662619219
-0.9503732342947624 0.6906078800378381
-0.5471296668243022 0.02304774986165942
-0.547129665473636 0.023047747625663376
-0.5471296681749686 0.023047752097655604
-0.5334189821998142 0.0003500377316367834
-0.5334189820314388 0.0003500374528954179
-0.5334189823681895 0.0003500380103782626
-0.5501820516788306 0.02810090035236401
-0.5501820531663681 0.02810090281494679
-0.5501820501912935 0.028100897889781404
-0.5332075543615314 2.397659474092398e-08
-0.5332075543601374 2.3974287460736487e-08
-0.5332075543629253 2.39789021616444e-08
-0.5332075398783308 1.3099330647969865e-15
-0.5332075398783311 1.907640169961089e-15
-0.5332075398783307 8.258638998365209e-16
Expectation value: -0.53320754 